In [6]:
from io import StringIO

import pandas as pd
import requests
from tqdm import tqdm

import config
from utils.mongo_controller import mongo_controller

data_dir = config.DATA_DIR
datasets_dir = data_dir / 'bob_usdt_datasets'

bolivian_blue_csv = datasets_dir / 'BolivianBlue.csv'
cripto_ya_csv = datasets_dir / 'CriptoYA-recent.csv'

In [21]:
# Import data from BolivianBlue.csv
# Read it to a pandas dataframe
bolivian_blue_df = pd.read_csv(bolivian_blue_csv)
# Convert all column 'Fecha' into datetime format. Original data is in format DD/MM/YY
bolivian_blue_df['Fecha'] = pd.to_datetime(bolivian_blue_df['Fecha'], format='%d/%m/%y')
# Localize data to America/La Paz
bolivian_blue_df['Fecha'] = bolivian_blue_df['Fecha'].dt.tz_localize('America/La_Paz')

for idx, row in tqdm(bolivian_blue_df.iterrows(), total=len(bolivian_blue_df), unit="doc",
                     desc="Inserting docs"):
    data_dict = {
        "timestamp": row['Fecha'],
        "source": "bolivianblue.com",
        "sell_price": row['Compra'],
        "buy_price": row['Venta']
    }
    exists = mongo_controller.query_data(_mode="one", collection="USDT_BOB_Other",
                                         _filter={"timestamp": data_dict["timestamp"],
                                                  "source": data_dict["source"]})
    if not exists:
        mongo_controller.save_data(collection="USDT_BOB_Other", data=data_dict)

100%|██████████| 43/43 [00:00<00:00, 44.85it/s]


In [7]:
# Import data from CriptoYA.csv
# Read it to a pandas dataframe
cripto_ya_df = pd.read_csv(cripto_ya_csv)
# Convert all column 'Fecha' into datetime format. Original data is in format YYYY-MM-DD
cripto_ya_df['Fecha'] = pd.to_datetime(cripto_ya_df['Fecha'])
# Localize data to America/La Paz
cripto_ya_df['Fecha'] = cripto_ya_df['Fecha']

for idx, row in tqdm(cripto_ya_df.iterrows(), total=len(cripto_ya_df), unit="doc",
                     desc="Inserting docs"):
    data_dict = {
        "timestamp": row['Fecha'],
        "source": "criptoya.com",
        "sell_price": row['Venta'],
        "buy_price": row['Compra']
    }
    exists = mongo_controller.query_data(_mode="one", collection="USDT_BOB_Other",
                                         _filter={"timestamp": data_dict["timestamp"],
                                                  "source": data_dict["source"]})
    if not exists:
        mongo_controller.save_data(collection="USDT_BOB_Other", data=data_dict)

Inserting docs: 100%|██████████| 44/44 [00:02<00:00, 20.04doc/s]


In [23]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vR2lRBAKrqBFtv_Y8glwaBq28banI80eg3wTOE9Y63LR8iVOjVhpxS3dpeBiqREYM3z1TgA0fdg_h7B/pub?output=csv"
response = requests.get(url)
csv_data = StringIO(response.text)
dolar_blue_bolivia_df = pd.read_csv(csv_data, header=None)

dolar_blue_bolivia_df = dolar_blue_bolivia_df.drop(dolar_blue_bolivia_df.columns[[1, 2]], axis=1)
dolar_blue_bolivia_df.columns = ['timestamp', 'buy', 'sell']

dolar_blue_bolivia_df['buy'] = dolar_blue_bolivia_df['buy'].str.replace(',', '.').astype(float)
dolar_blue_bolivia_df['sell'] = dolar_blue_bolivia_df['sell'].str.replace(',', '.').astype(float)

dolar_blue_bolivia_df['timestamp'] = pd.to_datetime(dolar_blue_bolivia_df['timestamp'], format='%d/%m/%Y %H:%M:%S')
dolar_blue_bolivia_df['timestamp'] = dolar_blue_bolivia_df['timestamp'].dt.tz_localize('America/La_Paz')

dolar_blue_bolivia_df = dolar_blue_bolivia_df[
    (dolar_blue_bolivia_df['timestamp'].dt.hour >= 7) & (dolar_blue_bolivia_df['timestamp'].dt.hour <= 23)]

for idx, row in tqdm(dolar_blue_bolivia_df.iterrows(), total=len(dolar_blue_bolivia_df), unit="doc",
                     desc="Inserting docs"):
    data_dict = {
        "timestamp": row['timestamp'],
        "source": "dolarbluebolivia.click",
        "sell_price": row['sell'],
        "buy_price": row['buy']
    }
    exists = mongo_controller.query_data(_mode="one", collection="USDT_BOB_Other",
                                         _filter={"timestamp": data_dict["timestamp"],
                                                  "source": data_dict["source"]})
    if not exists:
        mongo_controller.save_data(collection="USDT_BOB_Other", data=data_dict)

Inserting docs: 100%|██████████| 19101/19101 [12:59<00:00, 24.50doc/s]
